<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/ml/manual_two_layers_four_neurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 neural network with 3 inputs, 4 neurons in the first (hidden) layer, and 1 output neuron,

Use this data https://raw.githubusercontent.com/werowe/HypatiaAcademy/refs/heads/master/ml/log-neuron.txt


Input Layer (4) ---> [ 3 Neurons in Hidden Layer ] ---> Output Layer (1)


In [2]:
import numpy as np

#TODO write code





output Z layer 1 [-0.12897979 -1.2688535   0.31424232 -0.52109791]
activation a1 layer 1  [-0.12826929 -0.85348661  0.30429156 -0.47854694]

output Z layer 2 (ouput layer) [1.72690756]
activation a2 layer 2  0.8490164354238454

loss y=1 y_hat 0.8490164354238454

BACKPROPAGATION:
dW2: [ 0.01936655  0.12886245 -0.04594302  0.07225272]
db2: -0.15098356457615458
dW1: [[-0.01106553 -0.02655727  0.01549174]
 [ 0.03752818  0.09006762 -0.05253945]
 [ 0.01451156  0.03482775 -0.02031619]
 [-0.07543753 -0.18105006  0.10561254]]
db1: [-0.02213106  0.07505635  0.02902313 -0.15087505]


#Formulas for Computing Derivatives


## Forward Propagation

$
Z^{[1]} = W^{[1]}X + b^{[1]}
$

$
A^{[1]} = g^{[1]}(Z^{[1]})
$

$
Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}
$

$
A^{[2]} = g^{[2]}(Z^{[2]}) = \sigma(Z^{[2]})
$

---

## Backward Propagation

$
dZ^{[2]} = A^{[2]} - Y
$

$
dW^{[2]} = \frac{1}{m} \ dZ^{[2]} {A^{[1]}}^T
$

$
db^{[2]} = \frac{1}{m} \ \text{np.sum}(dZ^{[2]}, \text{axis}=1, \text{keepdims}=True)
$

$
dZ^{[1]} = W^{[2]^\top} dZ^{[2]} \odot g^{[1]'}(Z^{[1]})
$

$
dW^{[1]} = \frac{1}{m} dZ^{[1]} X^T
$

$
db^{[1]} = \frac{1}{m} \ \text{np.sum}(dZ^{[1]}, \text{axis}=1, \text{keepdims}=True)
$

